In [1]:
from datetime import timedelta

import numpy as np
import pandas as pd
import plotly.graph_objects as go

from Config import config
from data_processing.ohlcv import read_multi_timeframe_ohlcv
from helper.data_preparation import single_timeframe
from helper.helper import date_range_to_string

DEBUG@10-03.18:04:32:...Starting


In [29]:
import os

n_mt_ohlcv = pd.read_csv(os.path.join(r"C:\Code\dl-forcasting\data\Kucoin\Spot\BTCUSDT",
                               f"n_mt_ohlcv.24-03-01.00-00T24-09-01.00-00.csv.zip"), parse_dates=['date'], compression='zip')
config.processing_date_range = '24-03-01.00-00T24-09-01.00-00'
n_mt_ohlcv

,timeframe,date,open,close,high,low,volume
0,15min,2024-03-01 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN
1,1D,2024-03-01 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN
2,1h,2024-03-01 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN
3,1min,2024-03-01 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN
4,4h,2024-03-01 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
341347,1D,2024-09-01 00:00:00+00:00,-0.684385,-1.265078,-1.114403,-0.900461,0.298451
341348,1h,2024-09-01 00:00:00+00:00,-0.081138,-0.165222,-0.038223,-0.001254,-0.095335
341349,1min,2024-09-01 00:00:00+00:00,-0.018025,-0.275889,-0.122945,-0.169871,-0.171993
341350,4h,2024-09-01 00:00:00+00:00,-0.116954,-0.577484,-0.395726,-0.480249,0.044290


In [34]:
na_n_mt_ohlcv = n_mt_ohlcv[n_mt_ohlcv.isna().any(axis=1)]
end_of_nans = na_n_mt_ohlcv['date'].max()
end_of_nans

Timestamp('2024-05-20 00:00:00+0000', tz='UTC')

In [37]:
no_nan_n_mt_ohlcv = n_mt_ohlcv[ n_mt_ohlcv['date'] > end_of_nans]
no_nan_n_mt_ohlcv

,timeframe,date,open,close,high,low,volume
148418,1min,2024-05-20 00:01:00+00:00,-0.167760,-0.565577,-0.343654,-0.357794,-0.030649
148419,1min,2024-05-20 00:02:00+00:00,-0.507036,-0.611986,-0.654449,-0.479779,0.176402
148420,1min,2024-05-20 00:03:00+00:00,-0.723255,-1.421020,-0.531388,-1.244801,0.310774
148421,1min,2024-05-20 00:04:00+00:00,-1.398959,-0.977134,-1.174035,-1.022799,-0.218426
148422,1min,2024-05-20 00:05:00+00:00,-0.962370,-0.513302,-0.741274,-0.704919,-0.271016
...,...,...,...,...,...,...,...
341347,1D,2024-09-01 00:00:00+00:00,-0.684385,-1.265078,-1.114403,-0.900461,0.298451
341348,1h,2024-09-01 00:00:00+00:00,-0.081138,-0.165222,-0.038223,-0.001254,-0.095335
341349,1min,2024-09-01 00:00:00+00:00,-0.018025,-0.275889,-0.122945,-0.169871,-0.171993
341350,4h,2024-09-01 00:00:00+00:00,-0.116954,-0.577484,-0.395726,-0.480249,0.044290


In [38]:
start  = no_nan_n_mt_ohlcv['date'].min()
end  = no_nan_n_mt_ohlcv['date'].max()
duration_minutes = (end - start) / timedelta(minutes=1)
max_gap_minutes = 2880 # 2×24×60
(start , end, duration_minutes)

(Timestamp('2024-05-20 00:01:00+0000', tz='UTC'),
 Timestamp('2024-09-01 00:00:00+0000', tz='UTC'),
 149759.0)

In [39]:
np.random.seed(42)
simulated_gap_starts = start + np.random.uniform(0, duration_minutes - max_gap_minutes, 100) * timedelta(minutes=1) 
simulated_gap_durations = np.random.uniform(1, max_gap_minutes, 100) * timedelta(minutes=1)
simulated_gap_ends = simulated_gap_starts + simulated_gap_durations
list(zip(simulated_gap_starts, simulated_gap_ends, simulated_gap_durations))

[(Timestamp('2024-06-27 04:53:04.686971+0000', tz='UTC'),
  Timestamp('2024-06-27 06:24:33.764507+0000', tz='UTC'),
  datetime.timedelta(seconds=5489, microseconds=77536)),
 (Timestamp('2024-08-24 23:20:57.996671+0000', tz='UTC'),
  Timestamp('2024-08-26 05:54:11.531113+0000', tz='UTC'),
  datetime.timedelta(days=1, seconds=23593, microseconds=534442)),
 (Timestamp('2024-08-02 15:55:32.290759+0000', tz='UTC'),
  Timestamp('2024-08-03 07:01:34.142930+0000', tz='UTC'),
  datetime.timedelta(seconds=54361, microseconds=852171)),
 (Timestamp('2024-07-20 01:31:21.570023+0000', tz='UTC'),
  Timestamp('2024-07-21 01:56:32.071215+0000', tz='UTC'),
  datetime.timedelta(days=1, seconds=1510, microseconds=501192)),
 (Timestamp('2024-06-04 21:56:51.713373+0000', tz='UTC'),
  Timestamp('2024-06-06 17:30:44.746079+0000', tz='UTC'),
  datetime.timedelta(days=1, seconds=70433, microseconds=32706)),
 (Timestamp('2024-06-04 21:53:19.149148+0000', tz='UTC'),
  Timestamp('2024-06-05 09:52:01.888811+0000', 

In [40]:
no_nan_n_mt_ohlcv

,timeframe,date,open,close,high,low,volume
148418,1min,2024-05-20 00:01:00+00:00,-0.167760,-0.565577,-0.343654,-0.357794,-0.030649
148419,1min,2024-05-20 00:02:00+00:00,-0.507036,-0.611986,-0.654449,-0.479779,0.176402
148420,1min,2024-05-20 00:03:00+00:00,-0.723255,-1.421020,-0.531388,-1.244801,0.310774
148421,1min,2024-05-20 00:04:00+00:00,-1.398959,-0.977134,-1.174035,-1.022799,-0.218426
148422,1min,2024-05-20 00:05:00+00:00,-0.962370,-0.513302,-0.741274,-0.704919,-0.271016
...,...,...,...,...,...,...,...
341347,1D,2024-09-01 00:00:00+00:00,-0.684385,-1.265078,-1.114403,-0.900461,0.298451
341348,1h,2024-09-01 00:00:00+00:00,-0.081138,-0.165222,-0.038223,-0.001254,-0.095335
341349,1min,2024-09-01 00:00:00+00:00,-0.018025,-0.275889,-0.122945,-0.169871,-0.171993
341350,4h,2024-09-01 00:00:00+00:00,-0.116954,-0.577484,-0.395726,-0.480249,0.044290


In [41]:
gapped_n_mt_ohlcv = no_nan_n_mt_ohlcv.copy()
for start, end in zip(simulated_gap_starts, simulated_gap_ends):
        gapped_n_mt_ohlcv = gapped_n_mt_ohlcv[~((gapped_n_mt_ohlcv['date'] >= start) & (gapped_n_mt_ohlcv['date'] <= end))]
gapped_n_mt_ohlcv

,timeframe,date,open,close,high,low,volume
148418,1min,2024-05-20 00:01:00+00:00,-0.167760,-0.565577,-0.343654,-0.357794,-0.030649
148419,1min,2024-05-20 00:02:00+00:00,-0.507036,-0.611986,-0.654449,-0.479779,0.176402
148420,1min,2024-05-20 00:03:00+00:00,-0.723255,-1.421020,-0.531388,-1.244801,0.310774
148421,1min,2024-05-20 00:04:00+00:00,-1.398959,-0.977134,-1.174035,-1.022799,-0.218426
148422,1min,2024-05-20 00:05:00+00:00,-0.962370,-0.513302,-0.741274,-0.704919,-0.271016
...,...,...,...,...,...,...,...
341347,1D,2024-09-01 00:00:00+00:00,-0.684385,-1.265078,-1.114403,-0.900461,0.298451
341348,1h,2024-09-01 00:00:00+00:00,-0.081138,-0.165222,-0.038223,-0.001254,-0.095335
341349,1min,2024-09-01 00:00:00+00:00,-0.018025,-0.275889,-0.122945,-0.169871,-0.171993
341350,4h,2024-09-01 00:00:00+00:00,-0.116954,-0.577484,-0.395726,-0.480249,0.044290
